<a href="https://colab.research.google.com/github/Apofice2/Test-apprentissage-par-renforcement/blob/main/TP1_StickGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="./assets/Logo_ESEO_GROUPE.jpg" alt="Tech Logo" align="center" height="400" width="400"/>

<h1 align="center"; style="color:#3333cc;font-size:55px">Apprentissage par renforcement</h1>

<h2 align="center"; style="color:#0099cc;font-size:30px">TP1 : Développez un agent d'apprentissage par renforcement pour jouer au StickGame</h2>

Pour ce TP, vous implementerez un environnement de jeu de Stick Game ainsi qu'un agent capable d'exceller dans cet environnement. Utilisez ce notebook Jupyter qui passe en revue chaque partie du TP pour présenter des preuves et une analyse de vos résultats.

<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>📥 <strong>Import des librairies </strong></span></h2>

In [ ]:
from random import randint
import random
import numpy as np

<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>💾 <strong>Les règles du jeu</strong></span></h2>

<img src="./assets/stickgame.jpg" alt="Tech Logo" align="center" height="400" width="400"/>

Le jeu se joue à deux joueurs. Le joueur qui commence est tiré au sort. Le jeu commence avec un nombre prédéfini de batonnets (souvent 12 ou 16). Dans ce TP, nous commencerons le jeu avec 12 batonnets. Chaque joueur enlève au choix 1, 2 ou 3 bâtonnets puis c'est au tour de l'adversaire de retirer 1, 2 ou 3 bâtonnets. Celui qui enlèvera le dernier bâtonnet sera le perdant.

<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>🧠<strong>1. Formalisation du problème d'apprentissage par renforcement</strong></span></h2>

Un problème d'apprentissage par renforcement nécessite forcément des interactions entre un agent et un environnement.


<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  Quels sont les principales données que l'environnement doit fournir à l'agent ?</span></p>

In [ ]:
#l'état suivant et la récompenses suivante

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  En fonction de la règle de jeu du Stick Game énoncée précédemment, quelles valeurs peut-on attribuer à l'état et au récompenses renvoyées par l'environnement ? Quel est l'état initial ?</span></p>

In [ ]:
INITIAL_STATE = 12
FAIL_REWARD   =-1
GAME_IN_PROGRESS_REWARD = 0
WIN_REWARD = 1

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  Quelle est la principale donnée que l'agent fournit à l'environnement ?</span></p>

In [ ]:
#son état actuel et ses actions possibles.

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  En fonction de la règle de jeu du Stick Game énoncée précédemment, quelles sont les actions possibles dans l'espace des actions ? (Exprimez-les sous forme de liste Python)</span></p>

In [ ]:
ACTION_SPACE = [1,2,3]

<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>🌎 <strong>2. Conception de l'environnement</strong></span></h2>

L'environnement sera représenté sous forme d'une classe Python ayant des méthodes qui lui permettent d'interagir avec l'agent et l'utilisateur du notebook (c'est-à-dire vous).
Référez-vous directement aux docstrings de la classe pour comprendre l'utilité de chaque attribut et méthode de la classe

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ecrivez le code permettant de développer les méthodes de classe <strong>is_finished()</strong>, <strong>reset()</strong>, <strong>display()</strong>, <strong>step()</strong></span></p>

In [ ]:
class StickGame(object):
    '''
    Classe StickGame représentant l'environnement du jeu des batonnets

    '''
    def __init__(self, sticks):
        # Comme dans toutes les classes, la fonction __init__ est la fonction d'initialisation.
        # Cette fonction est appelée à chaque fois qu'un objet de cette classe est instanciée
        # Au moment de l'initialisation, les attributs de l'objet sont définis

        super(StickGame, self).__init__()
        # L'attribut original_nb est le nombre de batonnets au début de chaque partie.
        # Cet attribut n'est pas modifiée au cours du jeu et permet juste de ré-initialiser le jeu en fin de partie
        self.original_sticks = INITIAL_STATE

        # L'attribut nb est le nombre de batonnets à tout moment au cours de la partie.
        # Cet attribut est mis à jour à chaque fois qu'un joueur joue
        self.sticks = INITIAL_STATE

    def is_finished(self):

        '''
        Méthode permettant de vérifier si le jeu est terminé ou non

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe


                Returns:
                        Booléen : True si le jeu est terminé, False sinon
        '''
        return self.sticks <= 0


    def reset(self):

        '''
        Méthode permettant de ré-initialiser le jeu à la fin de chaque partie

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe


                Returns:
                        self.nb : Le nombre de batonnets au début de chaque partie selon la règle du jeu
        '''
        self.sticks = self.original_sticks
        return self.sticks


    def display(self):

        '''
        Méthode permettant d'afficher le nombre de batonnets disponibles dans le jeu
        après chaque action de jeu des joueurs

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe


                Returns:
                        Rien : affiche avec print le nombre de batonnets restant disponibles dans le jeu
        '''

        print ("| " * self.sticks)

    def step(self, action):

        '''
        Méthode permettant de renvoyer à l'agent après chaque de sa part le nouvel état de l'environnement
        ainsi que la récompense obtenue

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe
                        - action (int) : Nombre entier désignant l'action entreprise par l'agent


                Returns:
                        - current_state (int/None) : Nombre entier ou None (dans le cas où le jeu est terminé)
                                                     désignant l'état actuel du jeu
                        - reward (int) : Nombre entier désignant la récompense à fournir à l'agent
                                         en fonction de l'état du jeu

        '''
        self.sticks = self.sticks - action
        if(action not in ACTION_SPACE):
            print(ACTION_SPACE)
            raise Exception("Merci de choisir un nombre entre 1 et 3 svp !")
        else :
            if self.sticks <= 0 :
                reward = FAIL_REWARD
                current_state = None
            else :
                current_state = self.sticks
                reward = GAME_IN_PROGRESS_REWARD

            return current_state, reward


In [ ]:
class StickGame(object):
    '''
    Classe StickGame représentant l'environnement du jeu des batonnets

    '''

    def __init__(self, nb):
        # Comme dans toutes les classes, la fonction __init__ est la fonction d'initialisation.
        # Cette fonction est appelée à chaque fois qu'un objet de cette classe est instanciée
        # Au moment de l'initialisation, les attributs de l'objet sont définis

        super(StickGame, self).__init__()
        # L'attribut original_nb est le nombre de batonnets au début de chaque partie.
        # Cet attribut n'est pas modifiée au cours du jeu et permet juste de ré-initialiser le jeu en fin de partie
        self.original_nb = INITIAL_STATE

        # L'attribut nb est le nombre de batonnets à tout moment au cours de la partie.
        # Cet attribut est mis à jour à chaque fois qu'un joueur joue
        self.nb = INITIAL_STATE

    def is_finished(self):

        '''
        Méthode permettant de vérifier si le jeu est terminé ou non

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe


                Returns:
                        Booléen : True si le jeu est terminé, False sinon
        '''
        print(self.nb)
        print(type(self.nb))
        return self.nb <= 0

    def reset(self):

        '''
        Méthode permettant de ré-initialiser le jeu à la fin de chaque partie

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe


                Returns:
                        self.nb : Le nombre de batonnets au début de chaque partie selon la règle du jeu
        '''


        self.nb=self.original_nb
        return self.nb





    def display(self):

        '''
        Méthode permettant d'afficher le nombre de batonnets disponibles dans le jeu
        après chaque action de jeu des joueurs

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe


                Returns:
                        Rien : affiche avec print le nombre de batonnets restant disponibles dans le jeu
        '''

        print ("| " * self.nb)

    def step(self, action):

        '''
        Méthode permettant de renvoyer à l'agent après chaque action de sa part le nouvel état de l'environnement
        ainsi que la récompense obtenue

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe
                        - action (int) : Nombre entier désignant l'action entreprise par l'agent


                Returns:
                        - current_state (int/None) : Nombre entier ou None (dans le cas où le jeu est terminé)
                                                     désignant l'état actuel du jeu
                        - reward (int) : Nombre entier désignant la récompense à fournir à l'agent
                                         en fonction de l'état du jeu

        '''
        self.nb=self.nb-action
        reward=GAME_IN_PROGRESS_REWARD
        if self.is_finished()==True :
          reward=FAIL_REWARD
          self.nb=None
        return self.nb , reward













<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  A présent, vous pouvez tester votre environnement </span></p>

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Instanciez une partie avec 12 batonnets initiaux </span></p>

In [ ]:
jeu=StickGame(12)

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Affichez le jeu </span></p>

In [ ]:
jeu.display()

| | | | | | | | | | | | 


<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Faites l'action de retirer 2 batonnets  </span></p>

In [ ]:
jeu.step(2)

10
<class 'int'>


(10, 0)

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Affichez le jeu </span></p>

In [ ]:
jeu.display()

| | | | | | | | | | 


<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Faites l'action de retirer 3 batonnets  </span></p>

In [ ]:
jeu.step(3)

7
<class 'int'>


(7, 0)

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Affichez le jeu </span></p>

In [ ]:
jeu.display()

| | | | | | | 


<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ré-initialisez le jeu </span></p>

In [ ]:
jeu.reset()

12

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Affichez le jeu </span></p>

In [ ]:
jeu.display()

| | | | | | | | | | | | 


<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>🤖 <strong>3. Conception de l'agent</strong></span></h2>

L'agent est l'entité clé d'un problème d'apprentissage par renforcement. C'est lui qui décide des actions à entreprendre en fonction de l'état de l'environnement.

L'agent sera représenté sous forme d'une classe Python ayant des méthodes qui lui permettent d'interagir avec l'agent et l'utilisateur du notebook.

Référez-vous directement aux docstrings de la classe pour comprendre l'utilité de chaque attribut et méthode de la classe

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ecrivez le code permettant de développer les méthodes de classe <strong>reset_stat()</strong>, <strong>greedy_step()</strong>, <strong>play()</strong>, <strong>add_transition()</strong> et <strong>train()</strong> </span></p>

In [ ]:
from typing_extensions import Self
class StickPlayer(object):
    '''
    Classe StickPlayer représentant un agent jouant au jeu des batonnets

    '''

    def __init__(self, is_human, size, name, trainable=True):
        # Comme dans toutes les classes, la fonction __init__ est la fonction d'initialisation.
        # Cette fonction est appelée à chaque fois qu'un objet de cette classe est instanciée
        # Au moment de l'initialisation, les attributs de l'objet sont définis

        super(StickPlayer, self).__init__()

        # L'attribut name est une chaine de caractère qui représente le nom du joueur
        self.name = name

        # L'attribut is_human est un booléen qui définit si l'agent à instancier est un agent humain
        # ou un agent artificiel (IA)
        self.is_human = is_human

        # L'attribut history est une liste Python dans laquelle seront enregistrées les trajectoires du jeu
        # au cours des différentes parties jouées (épisodes)
        # Une trajectoire est un ensemble des transitions observées au cours d'une partie (épisode)
        # Une transition est un tuple (S, A, R, S') où
        #  - S  : est un l'état courant de l'environnement
        #  - A  : est l'action entreprise par l'agent
        #  - R  : est la récompense donnée par l'environnement suite à l'action A de l'agent
        #  - S' : est le nouvel état de l'environnement après que l'agent ait réalisé l'action A
        self.history = []

        # L'attribut V est un dictionnaire Python qui représente la fonction de valeur de l'agent.
        # Chaque clé du dictionnaire représente chacun des états de l'environnement
        # La valeur associée à chaque clé est la valeur de l'état, c'est-à-dire à quel point il est bon
        # pour l'agent d'être dans ce état
        self.V = {}

        # On initialise le dictionnaire avec toutes les valeurs d'état à 0.
        for s in range(1,size+1):
            self.V[s] = 0.

        # L'attribut win_nb est un nombre entier représentant le nombre de victoires cumulées de l'agent
        # sur un ensemble de parties jouées.
        # Cet attribut sera utilisé à des fins de statistiques pour évaluer l'agent au terme de son entrainement
        self.win_nb = 0.

        # L'attribut lose_nb est un nombre entier représentant le nombre de défaites cumulées de l'agent
        # sur un ensemble de parties jouées
        # Cet attribut sera utilisé à des fins de statistiques pour évaluer l'agent au terme de son entrainement
        self.lose_nb = 0.

        # L'attribut rewards est une liste Python dans laquelle seront l'ensemble des récompenses obtenues
        # tout au long des parties jouées
        # Cet attribut sera utilisé à des fins de statistiques pour évaluer l'agent au terme de son entrainement
        self.rewards = []

        # L'attribut eps (epsilon) est un nombre réel représentant le coefficient d'exploration initial de l'agent,
        # c'est-à-dire la tendance de l'agent à entreprendre des actions aléatoires au lieu d'entreprendre
        # des actions permettant d'atteindre l'état avec la plus grande valeur
        # Cette valeur diminuera au fur et à mesure au cours de l'entrainement
        self.eps = 0.99

        # L'attribut trainable est un booléen représentant si l'agent instancié peut être entrainé ou non.
        # Par exemple, si l'agent est un joueur humain, aucun entrainement n'est nécessaire.
        self.trainable = trainable

    def reset_stat(self):

        '''
        Méthode permettant de ré-initialiser les statistiques de victoires et défaites de l'agent

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe


                Returns:
                        Rien : ré-initialise à 0 les attributs win_nb, lose_nb, rewards.
        '''
        self.win_nb=0
        lose_nb=0
        rewards=0



    def greedy_step(self, state):

        '''
        Méthode permettant de déterminer l'action gloutonne (greedy) à entreprendre en fonction de l'état de l'environnement
        Une action gloutonne est une action d'exploitation basée sur la connaissance à priori de l'agent
        C'est-à-dire atteindre les états avec le plus de valeur.

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe
                        - state (int) : Nombre entier représentant l'état courant de l'environnement


                Returns:
                        - action (int) : Nombre entier représentant la meilleure action à entreprendre
        '''
        actions = ACTION_SPACE
        current_worse_value = None
        action_index = None

        for i in range(0,3):
          a = actions[i]

          if state - a > 0 and (current_worse_value is None or current_worse_value > self.V[state - a]):
            print("current_best_value_before : ", current_worse_value)

            current_worse_value = self.V[state - a]
            action_index = i
        return actions[action_index if action_index is not None else np.random.choice(range(len(actions)))]






    def play(self, state):

        '''
        Méthode qui détermine :
         - Pour l'agent artificiel : l'action que l'agent doit entreprendre et transmettre à
           l'environnement(action gloutonne ou exploratrice)
         - Pour l'agent humain : l'action directement rentrée au clavier

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe
                        - state (int) : Nombre entier représentant l'état courant de l'environnement


                Returns:
                        - action (int) : Nombre entier représentant la meilleure action à entreprendre
        '''
        a=1

        if self.is_human==True:
          while True:

            a=int(input())
            if a not in [1,2,3]:
              print("error input incorrect réfléchie avant d'agir")
            else:
              return a
              break


        else:
          randome_valeur=random.uniform(0, 1)
          if randome_valeur < self.eps:
            a=randint(1,3)
          else:

            a=self.greedy_step(state)

        return a

    def add_transition(self, n_tuple):

        '''
        Méthode qui rajoute une transition à (S, A, R, S') à l'historique  des transitions et
        qui rajoute la récompense obtenue à la liste des récompenses

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe
                        - n_tuple (tuple) : Tuple de 4 éléments (S, A, R, S')


                Returns:
                        - Rien : transition rajoutée à l'historique et récompense rajoutée à la liste des récompenses
        '''
        self.history.append(n_tuple)
        self.rewards.append(n_tuple[2])



    def train(self):

        '''
        Méthode qui réalise l'entrainement de l'agent, c'est-à-dire qui ajuste par itération successives
        la fonction des valeurs pour chaque état de l'environnement à partir des transitions enregistrées.

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe


                Returns:
                        - Rien : fonction de valeur mise à jour itérativement, puis historique vidé à la fin des itérations
        '''
        if self.trainable==False:
          return"....."

        else:

          for i in self.history:
            if i[2]== 0:

              self.V[i[0]]=self.V[i[0]]+0.02*(self.V[i[3]]-self.V[i[0]])
            else:
              self.V[i[0]]=self.V[i[0]]+0.02*(self.V[i[3]]-self.V[i[0]])
          self.history=[]





<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>🎮 <strong>4. Fonction de jeu</strong></span></h2>

La fonction de jeu permet de faire jouer deux joueurs au StickGame, d'enregistrer les transitions correspondant à chaque action des joueurs puis d'entrainer les deux agents si besoin. Les agents peuvent être aussi bien des agents artificiels que des agents humains

In [ ]:
def play_game(game, p1, p2, train=True):

    '''
        Fonction qui permet de faire jouer deux agents (humains ou artificiels) au StickGame, d'enregistrer les
        transitions dans l'historique des agents, puis de faire l'entrainement des agents si besoin.

                Parameters:
                        - game (objet StickGame): objet instancié de la classe StickGame qui est un environnement de jeu
                        - p1 (objet StickPlayer) : objet instancié de la classe StickPlayer qui est un agent artificiel ou humain
                        - p2 (objet StickPlayer) : objet instancié de la classe StickPlayer qui est un agent artificiel ou humain


                Returns:
                        - Rien
    '''

    print("[INFO] : Starting new game !")

    # Commençons par ré-initialiser l'environnement de jeu pour s'assurer que l'environnement est dans son état initial
    state = game.reset()

    # Définissons la liste des deux joueurs
    # Le joueur en premier dans la liste est celui qui commence à jouer
    players = [p1, p2]

    # On utilise la fonction shuffle pour "mélanger" l'ordre des joueurs dans liste et donc choisir aléatoirement
    # le joueur qui commence à jouer en premier dans la partie
    random.shuffle(players)

    # Ici, p est le nombre de tours dans une partie.
    # On l'initialise à 0 avant le début d'une partie
    p = 0

    # On commence ici la boucle de jeu.
    # La boucle ne s'interrompt que lorsque le jeu est terminé, c'est-à-dire lorsqu'il n'y a plus de batonnets en jeu
    while game.is_finished() is False:

        # Lorsque le joueur qui joue est un humain, on affiche l'état du jeu
        if players[p%2].is_human:
            game.display()

        # Ici p%2 va varier entre 0 et 1 suivant que p soit pair ou impair
        # Par exemple, à p=0, p%2=0 et donc c'est le premier joueur de la liste (players[0]) qui joue
        # A l'itération suivante, à p=1, p%2=1, et donc c'est le deuxième joueur de la liste (players[1]) qui joue
        # Le joueur joue : on récupère l'action qu'il choisit d'entreprendre
        action = players[p%2].play(state)

        # L'action entreprise est transmise à l'environnement,
        # qui en retour communique le nouvel état et la récompense obtenue
        n_state, reward = game.step(action)

        #  Lorsque la récompense n'est pas égale à 0, cela veut dire que la partie est terminée
        # On mets à jour les statistiques de victoires/défaites pour chaque joueur
        if (reward != GAME_IN_PROGRESS_REWARD):
            print("[INFO] : Game Over ! ")

            # Le joueur qui a joué le dernier coup a perdu
            players[p%2].lose_nb += 1. if reward == -1 else 0

            # Et l'autre joueur a gagné
            players[(p+1)%2].win_nb += 1. if reward == -1 else 0
            print("[INFO] : Player '{}' wins ! ".format(players[(p+1)%2].name))
            print("==="*15)


        # Add the reversed reward and the new state to the other player

        # Rappel : Pour chaque joueur, une transition n'est réalisée que lorsque l'adversaire a joué son coup.
        # En d'autres termes, l'état dans lequel il se retrouve suite à une action n'est calculée qu'après que l'adversaire ait joué
        # Cela est nécessaire dans l'entrainement de l'agent pour prendre en compte la stratégie de l'adversaire
        # C'est pour cela qu'à chaque coup d'un joueur, la dernière transition de l'autre joueur (history[-1])
        # doit être modifiée pour ne pas enregistrer l'état de l'environnement après son coup, mais après le coup de l'adversaire
        # On en profite également pour modifier la récompense qu'il a obtenu après son dernier coup
        # Sa récompense est alors l'inverse de la récompense du joueur qui joue actuellement
        # Lorsque le jeu est en cours, et que la récompense obtenue est 0, cela ne change rien (0 x -1 = 0)
        # A la fin du jeu, lorsque le joueur dont c'est le tour  a perdu, le joueur qui a joué son coup avant lui a gagné
        # Le joueur dont c'est le tour a la récompense -1, et le joueur qui a joué au tour précédent à la récompense +1
        if p != 0:
            s, a, r, sp = players[(p+1)%2].history[-1]
            players[(p+1)%2].history[-1] = (s, a, reward * -1, n_state)

        # Dans tous les cas, on enregistre partiellement la transition du joueur dont c'est le tour
        # L'état futur est None puisque l'état futur ne sera déterminé qu'après le tour de l'adversaire
        players[p%2].add_transition((state, action, reward, None))

        # On réalise la transition de l'état courant à l'état suivant
        state = n_state

        # On incrémente le compteur de coups
        p += 1

        # Et on continue la boucle tant que la partie n'est pas finie

    # Si l'argument train est activé (train=True), on ré-entraine les deux agents
    if train:
        p1.train()
        p2.train()

<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>👩‍🏫 <strong>5. Entrainement de l'agent</strong></span></h2>

L'environnement, l'agent et la fonction de jeu sont maintenant conçus.
Nous pouvons maintenant passer à l'entrainement de l'agent afin qu'il puisse apprendre à exceller au StickGame. L'entrainement consiste à trouver la fonction de valeur optimale permettant à l'agent de trouver à chaque instant l'état qui a le plus de valeur.

In [ ]:
N_EPISODES = 10

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ecrivez le code permettant d'entrainer un agent avec une stratégie "Duel d'agents" :
    <ul style="text-align: left; font-size: 16px; color:#0f767a">
    <li>Instancier un environnement de jeu StickGame</li>
    <li>Instancier deux agents IA </li>
    <li>Faites jouer les deux agents l'un contre l'autre pendant N_EPISODES parties</li>
    <li>Chaque 10 parties, diminuez le taux d'exploration de 0.4% sans jamais descendre en dessous de 0.05</li>
    <li>A la fin de chaque partie, entrainer les agents avec les transitions enregistrées</li>
  </ul> </span></p>

In [ ]:


game=StickGame(12)
p1=StickPlayer(False,INITIAL_STATE,"david",trainable = False)
p2=StickPlayer(False,INITIAL_STATE,"gerard",trainable = False)
#play_game(Game, p1, p2, train=True)

for i in range(N_EPISODES):
  play_game(game, p1, p2, train=True)
  if i%10:
    p1.eps=p1.eps-p1.eps*0.004
    p2.eps=p2.eps-p2.eps*0.004
  play_game(game, p1, p2, train=True)





[INFO] : Starting new game !
12
<class 'int'>
9
<class 'int'>
9
<class 'int'>
6
<class 'int'>
6
<class 'int'>
5
<class 'int'>
5
<class 'int'>
3
<class 'int'>
3
<class 'int'>
1
<class 'int'>
1
<class 'int'>
-2
<class 'int'>
[INFO] : Game Over ! 
[INFO] : Player 'david' wins ! 
None
<class 'NoneType'>


TypeError: ignored

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ecrivez le code permettant de calculer le taux de victoire de chacun des agents, trouvez le meilleur des deux agents puis ré-initialiser ces statistiques. </span></p>

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  Quel est l'état de l'environnement le plus avantageux ? En d'autres termes, dans quel état vaut-il mieux se trouver pour avoir plus de chance de gagner ?</span></p>

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  Quel est l'état de l'environnement le moins avantageux ? En d'autres termes, dans quel état vaut-il mieux éviter de se trouver pour avoir plus de chance de gagner ?</span></p>

In [ ]:
# Affichez le fonction de valeur


<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  Comparez les fonctions de valeurs des deux agents. Que remarquez-vous ?</span></p>

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  A partir de ces observations, quelle condition décide de la victoire de l'un ou l'autre des agents ?</span></p>

<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>🧪 <strong>6. Test de l'agent entrainé contre un agent "random"</strong></span></h2>

Dans cette partie, nous faisons jouer notre meilleur agent entrainé contre un agent artificiel "random", c'est-à-dire un agent qui ne prend que des décisions aléatoires.

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ecrivez le code permettant de faire jouer le meileur agent IA contre un agent "random" pendant N_EPISODES parties:
    <ul style="text-align: left; font-size: 16px; color:#0f767a">
    <li>Instancier un agent random</li>
    <li>Faire jouer l'agent random contre le meilleur agent artificiel entrainé pendant N_EPISODES parties</li>
  </ul>
    </span></p>

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ecrivez le code permettant de calculer le taux de victoire de chacun des deux agents
    </span></p>

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  Il est sureprenant de voir l'agent random gagner tout de même quelques parties. A quoi cela peut-il être dû ?</span></p>

<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>🧪 <strong>6. Test de l'agent entrainé contre un agent humain</strong></span></h2>

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ecrivez le code permettant de faire jouer le meileur agent IA contre un agent humain, c'est-à-dire vous:
    <ul style="text-align: left; font-size: 16px; color:#0f767a">
    <li>Instancier un agent humain</li>
    <li>Jouer en boucle contre l'agent IA entrainé</li>
  </ul>
    </span></p>